In [1]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install tqdm 
!pip install gpustat
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
from tqdm.notebook import tqdm
import time
import json

ModuleNotFoundError: No module named 'torch'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # the device to load the model onto

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", quantization_config=quantization_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [ ]:
revised_data = []
dataset = load_dataset("fka/awesome-chatgpt-prompts")

# Access the 'train' split
train_dataset = dataset['train']

# Extract prompts from the 'train' split
dataset_prompt = train_dataset['prompt']

dataset_prompt = dataset_prompt[:100]

def revision(principles_list):
    for prompt in tqdm(dataset_prompt):
        input_id = tokenizer.encode(prompt, return_tensors='pt')
        with torch.no_grad():
            output = model.generate(input_id, max_length=512, num_return_sequences=1, no_repeat_ngram_size=2,
                                    pad_token_id=tokenizer.eos_token_id)
        #base_answer = output[0][len(input_id[0]):].tolist()
        base_ans = output[0].tolist()
        base_answer = tokenizer.decode(base_ans, skip_special_tokens=True)
        original_ans = base_answer
        
        #print("prompt: ", prompt, "\n")
        #print("ans:\n", base_answer, "\n\n")
        #batch inference to be implemented
   
        for principle in principles_list:
            critique = f"Revise the following response with respect to {principle}: '{base_answer}'. Please be concise in your answer and try to answer in 500 tokens."
        # new_prompt = f"{critique}"
        # new_prompt = f"{prompt}\n{critique}"
            new_inputs = tokenizer.encode(critique, return_tensors='pt')
            with torch.no_grad():
                new_out = model.generate(new_inputs, max_length=1000, num_return_sequences=1, no_repeat_ngram_size=2,
                                         pad_token_id=tokenizer.eos_token_id)
            #crit_ans = new_out[0][len(new_inputs[0]):].tolist()
            criticized_answer = tokenizer.decode(new_out[0], skip_special_tokens=True)
            #print("Critique:\n ", criticized_answer, "\n")
            #rev_prompt = f"Revise the following response:\n '{base_answer}'\n and remove all the critiques listed in the following criticised answer:\n '{criticized_answer}'" 
            #rev_prompt1 = tokenizer.encode(rev_prompt, return_tensors='pt').to('cuda')
            #with torch.no_grad():
                #new_ans = model.generate(rev_prompt1, max_length=512,num_return_sequences=1, no_repeat_ngram_size=2,
                                    # pad_token_id=tokenizer.eos_token_id)
            #rev_a = new_ans[0][len(rev_prompt1[0]):].tolist()
            #revised_answer = tokenizer.decode(new_ans[0], skip_special_tokens=True)
            #print("rev_ans:\n ", revised_answer, "\n")
            base_answer = criticized_answer
        
        revised_data.append({
            "prompt": prompt,
            "revised_answer": base_answer,
        })
    
    with open("revised_datafile.json", "w") as f:
        json.dump(revised_data, f, indent=4)


In [ ]:
principles_list = ["honesty", "prudence"]
revision(principles_list)